In [7]:
import os
import glob
import shlex
import shutil

## os
> 操作文件和目录
>> getcwd()

>> chdir()

>> listdir()

>> pardir():获取当前目录的父目录字符串名

>> makedirs()：生成多层递归目录

>> removedirs()：删除多层递归空目录，若目录不为空则无法删除

>> mkdir()：生成单级目录

>> rmdir()：删除单级空目录，若目录不为空则无法删除，报错，需要用shutil.rmtree

>> remove()：删除一个文件。

> 路径相关
>> path.isfile()和path.isdir()：检验给出的路径是文件还是目录

>> path.exists()：检验给出的路径是否存在

>> path.split()：返回一个路径的目录名和文件名

>> path.abspath()：获得绝对路径

>> path.basename()：返回文件基名

>> path.getsize()：获得文件大小

>> path.getatime(),getctime()和getmtime()：时间戳形式，需要转换：datetime.datetime.fromtimestamp()

>> path.splitext()：分离文件名与扩展名，用于判断某一类型文件

>> path.join(path,name)：连接目录与文件名或目录

> 系统相关  
>> os.system()：运行shell命令

In [2]:
os.path.split("C:/Users/zuo_tianyu0101/Desktop/代理.txt")

('C:/Users/zuo_tianyu0101/Desktop', '代理.txt')

In [4]:
os.path.basename("C:/Users/zuo_tianyu0101/Desktop/代理.txt")

'代理.txt'

In [6]:
os.path.splitext("C:/Users/zuo_tianyu0101/Desktop/代理.txt")

('C:/Users/zuo_tianyu0101/Desktop/代理', '.txt')

In [ ]:
#自动关机 
os.system('shutdown -s -t 1') 

#重启 
os.system('shutdown -r')

## sys 
> argv：命令行参数List，第一个元素是程序本身路径

> version：获取Python解释程序的版本信息

> path：返回模块的搜索路径
>> 在脚本中额外指定模块路径 sys.path.append("/nextcode/nfs_test/users/jiawei_new/lib/python2.7/site-packages")

> platform                             

> exit(n)：退出程序
>> 正常退出时exit(0)

> getdefaultencoding()                   

> setdefaultencoding(name)               

> stderr：错误输出

> stdin：标准输入

> stdout：标准输出
>> stdout.write() 只支持输出字符串
>> stdout.flush() 手动刷新缓冲区，可以边迭代边输出

## glob
可以使用Unix shell风格的通配符匹配符合特定格式的文件和文件夹
> glob.glob(pathname, recursive=False)：返回匹配列表

In [ ]:
for _file in glob.glob("%s/*success" %(log_dir)):
    sample_name = re.search(r".*\.(.*-[NT])\..*",_file).group(1)
    finished_samples.append(sample_name)

## shlex
为shell语法编写词法分析器

In [ ]:
#使用类shell语法拆分字符串
shlex.split("grep -c success %s" %cmd)

## shutil
是一种高层次的文件操作工具,主要强大之处在于其对文件的复制与删除操作更是比较支持好
> copy(src, dst, *, follow_symlinks=True)
    
> copy2(src, dst, *, follow_symlinks=True)：在copy上的基础上最后访问时间与修改时间也复制过来了	
    
> copyfile(src, dst, *, follow_symlinks=True)

> copyfileobj(fsrc, fdst, length=16384)：用于下载网上资源

> copytree(src, dst, symlinks=False, ignore=None, ignore_dangling_symlinks=False)：迭代复制目录，dst必需存在

> move(src, dst)
    
> rmtree(path, ignore_errors=False, onerror=None)：可以删除非空目录（os.rmdir无法做到）

> unpack_archive(filename,extract_dir=None)：解压缩文件

> make_archive(base_name, format, root_dir=None)：压缩文件
>> base_name：压缩完文件的名字

>> format：bztar, gztar, tar, zip

>> root_dir：压缩文件的起始位置（默认当前目录）

## 示例 

### WGBS_clean.py 

In [ ]:
import os
import glob
import argparse
import re
import shutil

"""
p3 ~/script/methylation/file_clean.py -d /ehpcdata/zuotianyu/WGBS -ld 01task_bismark -k bam pe.CX_report.txt PE_report.txt
"""

process_dir = open("processed_dir","a+")

def parse_args():
    parser = argparse.ArgumentParser(description="保留对应的文件")
    parser.add_argument("-d",help="总目录")
    parser.add_argument("-ld",help="log文件夹目录")
    parser.add_argument("-k", type=str, nargs='+',help="保留文件")
    args = parser.parse_args()
    return args.d, args.ld, args.k

def clean_save(work_dir,log_dir,keep_files):
    os.chdir(work_dir)
    finished_samples = []
    for _file in glob.glob("%s/*success" %(log_dir)):
        sample_name = re.search(r".*\.(.*-[NT])\..*",_file).group(1)
        finished_samples.append(sample_name)
    for sample in finished_samples:
        os.chdir("Sample_%s" %(sample))
        process_dir.write(os.getcwd()+"\n")
        for _file in glob.glob("*"):
            if delete(_file,keep_files):
                pass
            else:
                try:
                    os.remove(_file)
                except Exception as e:
                    shutil.rmtree(_file, ignore_errors=True)
        os.chdir(work_dir)

def delete(_file,keep_files):
    for keep in keep_files:
        if _file.endswith(keep):
            return 1
        else:
            pass
    return

if __name__ == "__main__":
    work_dir,log_dir,keep_files = parse_args()
    clean_save(work_dir,log_dir,keep_files)